## **Upload model + upload image as photo.jpg**

In [ ]:
from IPython.core.display import clear_output
#@title import library and load model etc
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
from IPython.display import Image as ImageOpen
from IPython.display import display as IpyDisplay
from IPython.display import Audio
import os

#clear old files
!rm keras_model.h5
!rm labels.txt
!rm no_sign.wav
!rm yes_sign.wav


!wget https://github.com/lolamchan/potential-eureka/raw/main/converted_keras.zip
!wget https://github.com/lolamchan/potential-eureka/raw/main/soundtrack/no_sign.wav
!wget https://github.com/lolamchan/potential-eureka/raw/main/soundtrack/yes_sign.wav
!unzip converted_keras.zip
model = load_model('keras_model.h5')
!rm converted_keras.zip
clear_output()

def playsound(num):
  if num == 0:
    IpyDisplay(Audio('yes_sign.wav', autoplay=True))
  elif num == 1:
    IpyDisplay(Audio('no_sign.wav',autoplay=True))
  else:
    return #later

In [ ]:
#@title upload photo
from google.colab import files
import os
!rm photo.jpg #remove photo.jpg

try:
  uploaded = files.upload()
except BaseException as err:
  error = str(err)
  if error.startswith('RangeError'):
    print('Please etry with a smaller file or upload via file browser (please rename to photo.jpg) .')
  else:
    print(error)
    print('Please try again.')
else:
  dst ='photo.jpg' 
  os.rename(list(uploaded.keys())[0], dst)



rm: cannot remove 'photo.jpg': No such file or directory


Please upload via file browser or retry with a smaller file.


In [ ]:
#@title Black Magic
display(ImageOpen('/content/photo.jpg'))
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
image = Image.open('/content/photo.jpg')

size = (224, 224)
image = image.resize(size, Image.ANTIALIAS)     #May lead to deformation of image (But better than cropping to size) (better solution may exist)
image_array = np.asarray(image)
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
data[0] = normalized_image_array


Label = open("/content/labels.txt", "r")        #Crude way of tearing open the txt and pick out the label
LabelDict = {}
LabelContent = Label.readlines()
for x in range(len(LabelContent)):
  LabelContent[x] = LabelContent[x].replace('\n','')
  LabelDict[x]=  LabelContent[x][1:]


prediction = model.predict(data)
predClass = int(np.argmax(prediction,axis=1))

playsound(predClass)
print(f'Predict it\'s a{LabelDict.get(predClass)}')
for x in range(len(LabelContent)):
  print(f'{round(prediction[0][x]*100)} % to be {LabelDict.get(x)}')


## **Don't run script below if image is uploaded from local drive**

In [ ]:
#@title webcam code
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):


  #Me: I know python
  #My code: ...
  
  
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
#@title take picture
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(ImageOpen(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))